In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [4]:
# 1️⃣ Load dataset
aqi = pd.read_csv("C:/Users/arsha/Downloads/archive/chennai_daily.csv")

# 2️⃣ Filter Chennai
aqi = aqi[aqi['city'] == 'Chennai']
# aqi['date'] = pd.to_datetime(
#     aqi['date'],
#     format='%d-%m-%Y'
# )

aqi['date'] = pd.to_datetime(aqi['date'], format='%Y-%m-%d')
aqi.drop(columns=['aqi','aqi_category'],inplace=True)
print(aqi.shape)
aqi.head()


(3287, 8)


,city,date,pm25,pm10,no2,so2,co,o3
13148,Chennai,2015-01-01,224.223627,314.623460,43.732344,8.120949,0.670627,52.107131
13149,Chennai,2015-01-02,162.682796,192.850461,40.044309,12.486799,0.773530,19.868737
13150,Chennai,2015-01-03,68.969535,91.922127,74.319567,14.854908,1.134935,36.726152
13151,Chennai,2015-01-04,62.032392,152.872953,55.361067,14.107804,1.203100,79.417726
13152,Chennai,2015-01-05,140.701181,211.691495,57.187228,10.352680,0.858703,60.209988


In [5]:
weather = pd.read_csv('C:/Users/arsha/Downloads/archive/weather_daily_dataset.csv')
weather = weather.rename(columns={'time': 'date'})
weather['date'] = pd.to_datetime(
    weather['date'],
    format='%d-%m-%Y'
)
weather['date'] = pd.to_datetime(weather['date'], format='%Y-%m-%d')

print(weather.shape)
# print(weather['date'].dt.year.unique())
weather.head()


(3287, 3)


,date,temp,humidity
0,2015-01-01,25.1,87
1,2015-01-02,25.6,87
2,2015-01-03,25.3,90
3,2015-01-04,25.2,86
4,2015-01-05,24.8,83


In [ ]:
# aqi = aqi.sort_values('Datetime')
# weather = weather.sort_values('Datetime')
# print("AQI range:", aqi['Datetime'].min(), "to", aqi['Datetime'].max())
# print("Weather range:", weather['Datetime'].min(), "to", weather['Datetime'].max())

AQI range: 2015-01-01 01:00:00 to 2020-06-30 23:00:00
Weather range: 2015-01-01 00:00:00 to 2020-06-30 23:00:00


In [6]:
merged = pd.merge(
    aqi,
    weather[['date', 'temp', 'humidity']],
    on='date',
    how='inner'
)
# merged = merged.drop(columns=['Benzene', 'Toluene', 'Xylene','AQI','AQI_Bucket','NO','NOx','NH3'])
# merged = merged.dropna(subset=['PM2.5','PM10'])
print(merged.shape)
# print(merged[['PM2.5','PM10']].isnull().sum())
merged.head()

(3287, 10)


,city,date,pm25,pm10,no2,so2,co,o3,temp,humidity
0,Chennai,2015-01-01,224.223627,314.623460,43.732344,8.120949,0.670627,52.107131,25.1,87
1,Chennai,2015-01-02,162.682796,192.850461,40.044309,12.486799,0.773530,19.868737,25.6,87
2,Chennai,2015-01-03,68.969535,91.922127,74.319567,14.854908,1.134935,36.726152,25.3,90
3,Chennai,2015-01-04,62.032392,152.872953,55.361067,14.107804,1.203100,79.417726,25.2,86
4,Chennai,2015-01-05,140.701181,211.691495,57.187228,10.352680,0.858703,60.209988,24.8,83


In [7]:
import numpy as np

# CPCB breakpoints
breakpoints = {
    "pm25": [(0,30,0,50),(31,60,51,100),(61,90,101,200),(91,120,201,300),(121,250,301,400),(251,1000,401,500)],
    "pm10": [(0,50,0,50),(51,100,51,100),(101,250,101,200),(251,350,201,300),(351,430,301,400),(431,1000,401,500)],
    # "no2": [(0,40,0,50),(41,80,51,100),(81,180,101,200),(181,280,201,300),(281,400,301,400),(401,1000,401,500)],
    # "so2": [(0,40,0,50),(41,80,51,100),(81,380,101,200),(381,800,201,300),(801,1600,301,400),(1601,2000,401,500)],
    # "co": [(0,1,0,50),(1.1,2,51,100),(2.1,10,101,200),(10.1,17,201,300),(17.1,34,301,400),(34.1,50,401,500)],
    # "o3": [(0,50,0,50),(51,100,51,100),(101,168,101,200),(169,208,201,300),(209,748,301,400),(749,1000,401,500)]
}

def calculate_subindex(conc, pollutant):
    for bp_low, bp_high, i_low, i_high in breakpoints[pollutant]:
        if bp_low <= conc <= bp_high:
            return ((i_high - i_low)/(bp_high - bp_low)) * (conc - bp_low) + i_low
    return np.nan

# Calculate AQI
sub_indices = []

for pol in breakpoints.keys():
    sub_indices.append(merged[pol].apply(lambda x: calculate_subindex(x, pol)))

merged["aqi"] = np.max(sub_indices, axis=0)

# AQI Category
def aqi_category(aqi):
    if aqi <= 50:
        return "Good"
    elif aqi <= 100:
        return "Satisfactory"
    elif aqi <= 200:
        return "Moderate"
    elif aqi <= 300:
        return "Poor"
    elif aqi <= 400:
        return "Very Poor"
    else:
        return "Severe"

merged["aqi_category"] = merged["aqi"].apply(aqi_category)


In [8]:
# scaler = MinMaxScaler()

# gases = merged[['NO2', 'SO2', 'CO', 'O3']]
# gases_scaled = scaler.fit_transform(gases)

# gases_scaled = pd.DataFrame(
#     gases_scaled,
#     columns=['NO2_s', 'SO2_s', 'CO_s', 'O3_s']
# )
# merged['Gas_Index'] = (
#     0.59 * gases_scaled['CO_s'] +
#     0.32 * gases_scaled['O3_s'] +
#     0.06 * gases_scaled['NO2_s'] +
#     0.03 * gases_scaled['SO2_s']
# )

merged.head()

,city,date,pm25,pm10,no2,so2,co,o3,temp,humidity,aqi,aqi_category
0,Chennai,2015-01-01,224.223627,314.623460,43.732344,8.120949,0.670627,52.107131,25.1,87,380.218133,Very Poor
1,Chennai,2015-01-02,162.682796,192.850461,40.044309,12.486799,0.773530,19.868737,25.6,87,332.989123,Very Poor
2,Chennai,2015-01-03,68.969535,91.922127,74.319567,14.854908,1.134935,36.726152,25.3,90,128.206342,Moderate
3,Chennai,2015-01-04,62.032392,152.872953,55.361067,14.107804,1.203100,79.417726,25.2,86,135.465922,Moderate
4,Chennai,2015-01-05,140.701181,211.691495,57.187228,10.352680,0.858703,60.209988,24.8,83,316.119511,Very Poor


In [9]:
merged.to_csv("final_dataset.csv", index=False)
# merged['aqi_category'].value_counts()
